# 패키지 import & 하이퍼파라미터 세팅

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

from tqdm import tqdm
import numpy as np
import random
# 하이퍼파라미터 설정
batch_size = 128
epochs = 90
learning_rate = 0.01
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 전처리 및 로드

In [2]:
# 데이터 경로
root = "/kaggle/input/islvrc-2012-10-pecent-subset"

transform = transforms.Compose([
    transforms.Resize((227, 227)),  # AlexNet의 입력 크기(227x227)로 조정
    transforms.ToTensor(),
    # RGB, ImageNet 데이터셋의 평균과 표준편차를 기반으로 설정된 값
    # https://velog.io/@gcoh5589/0.485-0.456-0.406
    # 만약, 자체 데이터셋으로 학습할 경우, 이미지 RGB의 평균과 표준편차를 구해서 정규화 값을 변경하는 것이 좋다고 함
    # https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 
])

# # 전체 데이터셋 로드(ILSVRC)
# dataset = datasets.ImageFolder(root=root, transform=transform)

# # Train/Test , 8:2
# train_size = int(0.8 * len(dataset))
# test_size = len(dataset) - train_size
# trainset, testset = random_split(dataset, [train_size, test_size])

# # DataLoader 생성 
# # shuffle : 각 에폭마다 데이터가 학습되는 순서가 변경,test일 경우 False로 설정(평가의 일관성과 재현성) 
# # num_workers : dataset의 데이터를 gpu로 전송할 때 필요한 전처리를 수행할 때 사용하는 subprocess의 수
# trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
# testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# 전체 데이터셋 로드(CIFAR10)
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# 데이터셋 크기 확인
print(f"Train dataset size: {len(trainset)}")
print(f"Test dataset size: {len(testset)}")

# 첫 번째 배치 확인 (이미지와 라벨)
images, labels = next(iter(trainloader))
print(f"Batch image shape: {images.shape}")  # (batch_size, channels, height, width)
print(f"Batch labels: {labels[:5]}")  # 첫 5개 라벨 확인

  0%|          | 0.00/170M [00:00<?, ?B/s]

  0%|          | 65.5k/170M [00:00<04:31, 628kB/s]

  0%|          | 229k/170M [00:00<02:26, 1.16MB/s]

  1%|          | 918k/170M [00:00<00:47, 3.55MB/s]

  2%|▏         | 3.70M/170M [00:00<00:13, 12.3MB/s]

...

100%|██████████| 170M/170M [00:03<00:00, 49.3MB/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


Files already downloaded and verified


Train dataset size: 50000
Test dataset size: 10000


Batch image shape: torch.Size([128, 3, 227, 227])
Batch labels: tensor([6, 8, 4, 6, 3])


# AlexNet 모델 정의

In [3]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000, dropout: float = 0.5) -> None:
        super(AlexNet, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0), # 1번 레이어 (48x2)
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(alpha=1e-4, beta=0.75, k=2, size=5), # LRN
            nn.MaxPool2d(kernel_size=3, stride=2), # stride < kernel_size일 경우 overlapping pooling
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2), # 2번 레이어 (128x2)
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(alpha=1e-4, beta=0.75, k=2, size=5), # LRN
            nn.MaxPool2d(kernel_size=3, stride=2), # overlapping pooling
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1), # 3번 레이어 (192x2)
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1), # 4번 레이어 (192x2)
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1), # 5번 레이어 (128x2)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2) # overlapping pooling
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6)) # 내부적으로 적절한 kernel size와 stride 값을 계산하여 풀링을 수행
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout), # dropout : 0.5
            nn.Linear(256 * 6 * 6, 4096), # 6번 레이어
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),# dropout : 0.5
            nn.Linear(4096, 4096), # 7번 레이어
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes) # 8번 레이어
        )
    #     self._init_bias() # 가중치 초기화
        
    # def _init_bias(self):
    #     for layer in self.layers:
    #         if isinstance(layer, nn.Conv2d):
    #             # 편차를 0.01로 하는 zero-mean 가우시안 정규 분포를 모든 레이어의 weight를 초기화
    #             # neuron bias: 2, 4, 5번째 convolution 레이어와 fully-connected 레이어에 상수 1로 적용하고 이외 레이어는 0을 적용
    #             nn.init.normal_(layer.weight, mean=0, std=0.01) # nn.init.normal_(input,mean=0.0,std=0.1) : input tensor를 N(mean,str^2)의 정규분포에 따라 초기화
    #             nn.init.constant_(layer.bias, 0) # nn.init.constant_(input,val) : input tensor를 val값으로 채운다.

    #     nn.init.constant_(self.layers[4].bias, 1) # CNN 2번 레이어 (128x2)
    #     nn.init.constant_(self.layers[10].bias, 1) # CNN 4번 레이어 (192x2)
    #     nn.init.constant_(self.layers[12].bias, 1) # CNN 5번 레이어 (128x2)
    #     nn.init.constant_(self.classifier[1].bias, 1) # MLP 6번 레이어
    #     nn.init.constant_(self.classifier[4].bias, 1) # MLP 7번 레이어
    #     nn.init.constant_(self.classifier[6].bias, 1) # MLP 8번 레이어
        
    def forward(self, x):
        x = self.layers(x) # CNN
        x = self.avgpool(x) # FCN의 입력 이미지 shape이 항상 6x6이 되는 것을 보장
        x = torch.flatten(x, 1) # FCN input
        x = self.classifier(x) # FCN
        return x

## AlexNet 구조 확인

In [4]:
import torchsummary
model = AlexNet(num_classes = 10).to(device) # 데이터 셋의 클래스 수 : 10개

# AlexNet의 Image 입력 사이즈는 (3, 227, 227), 원래 논문에서는 3 x 224 x 224가 맞지만 오류인 것으로 알려졌고 ,구조상 227이 맞다고 함 
torchsummary.summary(model, input_size=(3, 227, 227), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
 LocalResponseNorm-3           [-1, 96, 55, 55]               0
         MaxPool2d-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 27, 27]         614,656
              ReLU-6          [-1, 256, 27, 27]               0
 LocalResponseNorm-7          [-1, 256, 27, 27]               0
         MaxPool2d-8          [-1, 256, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         885,120
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 384, 13, 13]       1,327,488
             ReLU-12          [-1, 384, 13, 13]               0
           Conv2d-13          [-1, 256, 13, 13]         884,992
             ReLU-14          [-1, 256,

# 

# 손실 함수, 옵티마이저, lr scheduler 설정

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(momentum=MOMENTUM, weight_decay=WEIGHT_DECAY, params=model.parameters(), lr=learning_rate)

# adjust learning rate: validation error가 현재 lr로 더 이상 개선 안되면 lr을 10으로 나눠줌. 논문에서는 0.01을 lr 초기 값으로 총 3번 줄어듦
# factor : lr 업데이트 시 lr = lr*factor로 계산됨
# patience : 검증 손실값이 개선되지 않은 상태를 얼마나 허용할 것인지를 설정하는 정수값. 기본값은 10으로, 10번의 연속적인 epoch 동안 검증 손실값이 개선되지 않으면 learning rate를 감소시킴
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, factor=0.1, verbose=True, patience=4) # used if valid error doesn't improve.

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


# train

In [6]:
def train(model, data_loader, loss_fn, optimizer, device):
    model.train()
    running_loss = 0.0
    corr = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        
        # 누적 Gradient 초기화
        optimizer.zero_grad()

        # forward
        outputs = model(images)

        # loss
        loss_fn = criterion(outputs, labels)
        # backward
        loss_fn.backward()
        # Gradient 업데이트 
        optimizer.step()
        # 누적 loss 업데이트
        running_loss += loss_fn.item()*images.size(0) # 배치 전체 loss 계산
        # max probability와 max index를 반환
        # max index만 사용해서 실제 정답과 비교
        _, pred = torch.max(outputs, 1)
        # 정확히 맞춘 label의 합계를 계산
        corr += pred.eq(labels).sum().item()
    
    # 누적된 정답수를 전체 개수로 나누어 전체 정확도 계산
    acc = corr / len(data_loader.dataset)
    
    # 전체 loss와 전체 accuracy 반환
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

# test

In [7]:
# 테스트 루프
def test(model, data_loader, loss_fn, device):
    model.eval()
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수
        corr = 0.0
        running_loss = 0.0
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            corr += (predicted == labels).sum().item()
            running_loss += loss_fn(outputs, labels).item() * images.size(0)
    acc = corr / len(data_loader.dataset)
    return running_loss / len(data_loader.dataset), acc

# Evaluation

In [8]:
min_loss = np.inf
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
# Epoch 별 train & test
for epoch in tqdm(range(epochs)):
    # Model Training
    # train result
    train_loss, train_acc = train(model, trainloader, criterion, optimizer, device)

    # validation result
    val_loss, val_acc = test(model, testloader, criterion, device)   
    
    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        # torch.save(model.state_dict(), f'{MODEL_NAME}.pth')

    # learning rate update
    lr_scheduler.step(metrics=val_loss)
    
    # Epoch 별 결과
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')
    # 결과 저장
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

  0%|          | 0/90 [00:00<?, ?it/s]

  1%|          | 1/90 [01:22<2:01:46, 82.10s/it]

[INFO] val_loss has been improved from inf to 1.83192. Saving Model!
epoch 01, loss: 2.16399, acc: 0.18482, val_loss: 1.83192, val_accuracy: 0.31550


  2%|▏         | 2/90 [02:44<2:00:17, 82.01s/it]

[INFO] val_loss has been improved from 1.83192 to 1.38572. Saving Model!
epoch 02, loss: 1.61961, acc: 0.40258, val_loss: 1.38572, val_accuracy: 0.48980


...

 30%|███       | 27/90 [37:05<1:26:51, 82.73s/it]

epoch 27, loss: 0.02173, acc: 0.99382, val_loss: 0.55804, val_accuracy: 0.86870


 31%|███       | 28/90 [38:27<1:25:07, 82.38s/it]

epoch 28, loss: 0.02179, acc: 0.99394, val_loss: 0.56151, val_accuracy: 0.86900


 32%|███▏      | 29/90 [39:49<1:23:50, 82.47s/it]

# train/valid acc, loss 시각화

In [ ]:
import matplotlib.pyplot as plt
def plot_training_history(train_losses, val_losses, train_accuracies, val_accuracies):
    epochs = len(train_losses)

    # Loss
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(range(epochs), train_losses, label='Train Loss', color='blue')
    plt.plot(range(epochs), val_losses, label='Validation Loss', color='orange')
    plt.title('Train vs Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(range(epochs), train_accuracies, label='Train Accuracy', color='blue')
    plt.plot(range(epochs), val_accuracies, label='Validation Accuracy', color='orange')
    plt.title('Train vs Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training_history(train_losses, val_losses, train_accuracies, val_accuracies)